In [8]:
import re
from pathlib import Path
import os
import hjson
import numpy as np
import pandas as pd
from IPython.display import display
from loguru import logger
import time
import re

# Visualization
from phd_visualizations.test_timeseries import experimental_results_plot
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# Setup environment for running MATLAB code if not done externally
# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'
output_path: Path = Path(f'{os.getenv("HOME")}/development_psa/models_psa/docs/attachments')

# Debug file
debug_file = src_path / "Eurecat/MODEL_ERRORS/historic_dqn.xlsx"

# Resample figures using plotly_resampler
resample_figures = False

# sample_rate = '60s'
# sample_rate_numeric = int(sample_rate[:-1])

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Load configuration

with open( Path("data") / "variables_config.hjson") as f:
    vars_config = hjson.load(f)

# Update plot config
with open( Path("data") / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)


In [15]:
# Read states

from models_psa import calculate_benefits

# Index column is the first one, drop it
df = pd.read_excel(debug_file, index_col=0) 

# Get sample rate
sample_rate_numeric = df['sample_time'][0]

# Create new columns that start with qmed*, qsf, qts*, qhx*, q3wv* to q*
# This should've been handled when exporting the data, model.to_dataframe(rename_flows=True) or if not, now it will export both options
for col in df.columns:
    # logger.debug(f"Processing column {col}")
    if re.match(r'mmed*', col) or re.match(r'msf*', col) or re.match(r'mhx*', col) or re.match(r'm3wv*', col) or re.match(r'mts*', col):
        # logger.debug(f"Column {col} matches, naming: q{col[1:]}")
        df[f'q{col[1:]}'] = df[col]

# Just once, rename Pts_out to Pts_dis, it's already been fixed in the model
df.rename(columns={'Pts_out': 'Pts_dis'}, inplace=True)


df = calculate_benefits(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric)

In [16]:
# Visualize provided states

fig = experimental_results_plot(plt_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solarMED_debugging_{debug_file.stem}')
)

2024-04-04 12:47:17.460 | INFO     | phd_visualizations.test_timeseries:experimental_results_plot:378 - Optimization updates not shown in plot, show_optimization_updates: false
2024-04-04 12:47:17.461 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tamb
2024-04-04 12:47:17.461 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>amb</sub>
2024-04-04 12:47:17.462 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>amb</sub> added in yaxis1 (left), row 1, uncertainty: False, comparison: False
2024-04-04 12:47:17.464 | DEBUG    | phd_visualizations.test_timeseries:add_trace:38 - Attempting to add Tmed_c_in
2024-04-04 12:47:17.465 | DEBUG    | phd_visualizations.test_timeseries:add_trace:75 - legend_id: global_legend, legend: legend for trace T<sub>med,c,in</sub> (⁰C)
2024-04-04 12:47:17.467 | INFO     | phd_visualizations.test_timeseries:add_trace:288 - Trace T<sub>m

KeyError: 'Tts_h_out'

In [ ]:
# Reproduce

# Test model
from models_psa.solar_med import SolarMED

%autoreload 2

idx_start = 20
span = 12
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    sample_time=sample_rate_numeric,
    curve_fits_path='data/curve_fits.json',
    resolution_mode='simple',
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)

# Run model
# %autoreload 2

for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    
    logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qhx_s'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end}, elapsed time: {time.time()-start_time:.2f} seconds. Current operation state is {model.operating_state.name}")

    df_mod = model.to_dataframe(df_mod, rename_flows=True)
    

In [ ]:
# Visualize both together to see if they match